# Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

c:\Users\user\anaconda3\envs\LDA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
tweet = pd.read_csv('../../src/cleaned_mb.csv')
tweet = tweet.dropna(subset=['full_text'])
tweet = tweet[['created_at', 'full_text']]
tweet['created_at'] = pd.to_datetime(tweet['created_at'])
tweet

,created_at,full_text
0,2019-11-01 02:51:51+00:00,kerja ajar senang senang penuh semangat merdeka
1,2019-11-02 04:27:12+00:00,tpn guruguru merdeka ajar moga tahun depan gab...
2,2019-11-03 10:46:27+00:00,kbm ambengan batu admin mau share beberapa dok...
3,2019-11-03 10:50:12+00:00,kbm tl ambengan admin mau share beberapa dokum...
4,2019-11-03 14:43:27+00:00,harus ajar loyal nurani pikir sendiri my heart...
...,...,...
34593,2024-04-29 13:43:31+00:00,wakatobi siap gelar konferensi internasional s...
34594,2024-04-29 14:23:23+00:00,konsorsium baterai listrik nasional
34595,2024-04-29 15:50:14+00:00,joki essay tema merdeka ajar
34596,2024-04-29 18:23:19+00:00,tim pkm pnup terima dana pkm vokasi tahun


# Preprocess

In [3]:
tweet_quarter = tweet.copy()

tweet_quarter['quarter'] = tweet_quarter['created_at'].dt.to_period('Q')

tweet_quarter = tweet_quarter.groupby('quarter')

# Modeling

In [4]:
results = {}

In [5]:
%%time
for period, group in tweet_quarter:
    topic_model = BERTopic(verbose=False,
                           language='indonesian', calculate_probabilities=True,
                           nr_topics=10,
                           )
    text_data = group['full_text'].tolist()
    timestamps = group['created_at'].tolist()
    topics, probs = topic_model.fit_transform(text_data)
    results[period] = {
        'model' : topic_model,
        'topics_over_time' : topic_model.topics_over_time(text_data, timestamps, nr_bins=30),
        'probs': probs,
        'topic_info': topic_model.get_topic_info(),
        
        'topics': topics,
        'text': text_data,
    }

CPU times: total: 16min 37s
Wall time: 9min 14s


In [10]:
for period, result in results.items():
    print(f"Period: {period}")
    display(result['model'].get_topic_info())
    print()

Period: 2019Q4


,Topic,Count,Name,Representation,Representative_Docs
0,-1,315,-1_ajar_merdeka_nadiem_guru,"[ajar, merdeka, nadiem, guru, mendikbud, makar...",[hapus un nadiem siap program merdeka ajar gur...
1,0,457,0_guru_ajar_merdeka_hari,"[guru, ajar, merdeka, hari, program, selamat, ...",[hari guru nasional nadiem merdeka ajar guru g...
2,1,240,1_indonesia_guru_hari_selamat,"[indonesia, guru, hari, selamat, maju, gerak, ...",[selamat hari guru nasional guru gerak indones...
3,2,181,2_aku_merdeka_ajar_nadiem,"[aku, merdeka, ajar, nadiem, datang, ubah, rev...",[aku datang aku ajar aku uji aku revisi aku me...
4,3,74,3_menteri_mas_budaya_nadiem,"[menteri, mas, budaya, nadiem, ajar, merdeka, ...",[menteri budaya mendikbud nadiem anwar makari...
5,4,47,4_pokok_empat_tetap_mendikbud,"[pokok, empat, tetap, mendikbud, merdeka, ajar...","[mendikbud tetap empat pokok merdeka ajar, m..."
6,5,42,5_uji_nasional_hapus_asesmen,"[uji, nasional, hapus, asesmen, karakter, komp...",[menteri budaya nadiem makarim sampai empat p...
7,6,13,6_baik_sendiri_manusia_loyal,"[baik, sendiri, manusia, loyal, negara, enggak...",[aneh kebodohanterlalu fanatik manusiasehingga...
8,7,13,7_video_tv_dpr_komisi,"[video, tv, dpr, komisi, des, trans, youtube, ...",[besok hari rabu tanggal des jam mata najwa tr...
9,8,11,8_ppdb_zonasi_tetap_jalur,"[ppdb, zonasi, tetap, jalur, daerah, sistem, a...",[ppdb tetap sistem zonasi kuota jalur prestasi...



Period: 2020Q1


,Topic,Count,Name,Representation,Representative_Docs
0,-1,577,-1_ajar_merdeka_program_nadiem,"[ajar, merdeka, program, nadiem, guru, sekolah...",[mendikbud nadiem makarim luncur empat merdek...
1,0,438,0_merdeka_ajar_kampus_guru,"[merdeka, ajar, kampus, guru, sekolah, nadiem,...","[merdeka ajar kampus merdeka, merdeka ajar kam..."
2,1,370,1_ajar_merdeka_yuk_nadiem,"[ajar, merdeka, yuk, nadiem, episode, ikut, si...","[merdeka ajar ajar merdeka, merdeka ajar, merd..."
3,2,114,2_indonesia_merdeka_ajar_cikal,"[indonesia, merdeka, ajar, cikal, guru, kerja,...",[merdeka kerja sama kampus guru cikal kampus g...
4,3,22,3_aku_menang_revisi_datang,"[aku, menang, revisi, datang, uji, merdeka, aj...",[aku datang aku ajar aku uji aku revisi aku me...
5,4,17,4_teknologi_jalur_digital_online,"[teknologi, jalur, digital, online, internet, ...",[guru zaman millenial guna kipinpto giat evalu...
6,5,16,5_edar_surat_terbit_nomor,"[edar, surat, terbit, nomor, kemendikbud, tang...",[kemendikbud terbit surat edar nomor merdeka ...
7,6,15,6_selamat_jam_januari_wib,"[selamat, jam, januari, wib, kue, pukul, tahun...",[pagi kalian kunjung perlu perhati jam layan b...
8,7,14,7_peluang_negara_luar_negeri,"[peluang, negara, luar, negeri, kerja, buka, k...","[merdeka ajar buka peluang kerja sama negara, ..."
9,8,13,8_pakai_ukm_haram_internasional,"[pakai, ukm, haram, internasional, kibijakan, ...",[kibijakan merdeka ajar gagal kalau juta ukm t...



Period: 2020Q2


...

,Topic,Count,Name,Representation,Representative_Docs
0,-1,551,-1_ajar_merdeka_kurikulum_guru,"[ajar, merdeka, kurikulum, guru, indonesia, sa...",[ingat hari guru nasional jadi ruang apresiasi...
1,0,1091,0_guru_hari_nasional_gerak,"[guru, hari, nasional, gerak, selamat, raya, s...",[selamat hari guru nasional gerak sama raya me...
2,1,432,1_merdeka_ajar_kampus_kurikulum,"[merdeka, ajar, kampus, kurikulum, program, ma...",[hai sobat bsi rangka implementasi merdeka aj...
3,2,257,2_budaya_seni_salam_kayana,"[budaya, seni, salam, kayana, indonesia, pukul...",[kayana yuk saksi seru tayang ragam budaya ind...
4,3,52,3_kiamat_palestina_islam_palestine,"[kiamat, palestina, islam, palestine, agama, m...",[simak kunci jawab soal sumatif mata ajar mape...
5,4,32,4_bahasa_kosakata_yuk_satu,"[bahasa, kosakata, yuk, satu, revitalisasi, mb...",[halo satu bahasa daerah revitalisasi kalimant...
6,5,26,5_sejarah_enggak_dulu_ajar,"[sejarah, enggak, dulu, ajar, merdeka, rakyat,...",[kesi moyang mati lawan british nak merdeka de...
7,6,13,6_natal_damai_moga_selalu,"[natal, damai, moga, selalu, selamat, desember...",[segenap pimpin civitas akademika politeknik k...
8,7,11,7_kolaborasi_bagi_jambi_hepi,"[kolaborasi, bagi, jambi, hepi, untirta, kasim...","[giat bagi kolaborasi luring negeri kasimbar, ..."
9,8,11,8_harga_madiun_sekolah_raih,"[harga, madiun, sekolah, raih, baik, negeri, k...",[madiun terima harga kabupaten baik implementa...



Period: 2024Q1


,Topic,Count,Name,Representation,Representative_Docs
0,-1,500,-1_ajar_merdeka_program_sekolah,"[ajar, merdeka, program, sekolah, gera, kuriku...",[lang sama gera merdeka ajar wujud harap masa ...
1,0,226,0_kurikulum_guru_ajar_enggak,"[kurikulum, guru, ajar, enggak, merdeka, siswa...",[yaallah jujur pas ajar kurikulum gue benar sa...
2,1,212,1_indonesia_gera_ajar_merdeka,"[indonesia, gera, ajar, merdeka, harap, maju, ...",[gera merdeka ajar indonesia terus lebih tingk...
3,2,160,2_ajar_merdeka_gera_harap,"[ajar, merdeka, gera, harap, depan, lebih, mas...",[gera merdeka ajar memang jadi harap besar mas...
4,3,154,3_kampus_program_mahasiswa_merdeka,"[kampus, program, mahasiswa, merdeka, mbkm, aj...",[terus komitmen dukung program prioritas salah...
5,4,91,4_merdeka_ajar_harap_gera,"[merdeka, ajar, harap, gera, depan, kedepannya...","[gera merdeka ajar harap merdeka ajar depan, g..."
6,5,79,5_yuk_simak_hari_konten,"[yuk, simak, hari, konten, sama, tampil, bahas...",[konten kosakata panjang bahasa sasak samawa m...
7,6,59,6_seni_salam_budaya_isbi,"[seni, salam, budaya, isbi, bandung, wargi, ba...",[yuk kenal fakultas budaya media fbm isbi band...
8,7,38,7_tahun_klaten_ganjar_selamat,"[tahun, klaten, ganjar, selamat, warga, hari, ...",[ganjar pranowo sapa warga klaten hadir pesta ...
9,8,29,8_video_lomba_youtube_demokrasi,"[video, lomba, youtube, demokrasi, merdeka, ro...",[beri hadiah lomba video merdeka ajar selamat ...



Period: 2024Q2


,Topic,Count,Name,Representation,Representative_Docs
0,-1,69,-1_merdeka_ajar_ikut_daftar,"[merdeka, ajar, ikut, daftar, hadir, buat, kur...",[um fe kalo isi jurnal hari merdeka ajar siaka...
1,0,67,0_yuk_sama_hari_simak,"[yuk, sama, hari, simak, lebaran, buku, merdek...",[hari tampil kutip sekaligus biografi singkat ...
2,1,17,1_jawab_post_test_platform,"[jawab, post, test, platform, modul, kunci, aj...",[kunci jawab post test merdeka ajar modul ant...
3,2,33,2_indonesia_merdeka_ajar_negara,"[indonesia, merdeka, ajar, negara, dunia, seja...",[jalan panjang buah inovasi bidang bawa indon...
4,3,13,3_kampus_mahasiswa_merdeka_angkat,"[kampus, mahasiswa, merdeka, angkat, jadi, tuk...",[oif umsu terima kunjung tukar mahasiswa merde...
5,4,19,4_merdeka_ajar_gua_dunia,"[merdeka, ajar, gua, dunia, bro, guncang, wing...","[merdeka ajar, merdeka ajar, sama tahir viral ..."
6,5,57,5_ajar_merdeka_kurikulum_enggak,"[ajar, merdeka, kurikulum, enggak, benar, jadi...","[sih masuk kurikulum merdeka ajar, hasil kurik..."


In [8]:
for period, result in results.items():
    print(f"Period: {period}")
    print(result['model'].get_topic(0))
    print()

Period: 2019Q4
[('guru', 0.08517694201384238), ('ajar', 0.07364131219110379), ('merdeka', 0.06528279943316187), ('hari', 0.05180345093853399), ('program', 0.040913263195561066), ('selamat', 0.04031709486936009), ('nadiem', 0.0346369289650132), ('mendikbud', 0.03244633339491456), ('gerak', 0.0320838924990028), ('nasional', 0.03189649198685264)]

Period: 2020Q1
[('merdeka', 0.0723251950186973), ('ajar', 0.06855085761842135), ('kampus', 0.05866191780384118), ('guru', 0.05101645421165938), ('sekolah', 0.034015855768612664), ('nadiem', 0.03303618127965293), ('menteri', 0.030536673167625834), ('mendikbud', 0.029916015979020574), ('tinggi', 0.0218350156866603), ('program', 0.021602058868384746)]

Period: 2020Q2
[('merdeka', 0.08482326237000973), ('ajar', 0.07579508587475207), ('guru', 0.07458300842941852), ('kampus', 0.04839906898263946), ('jadi', 0.03762834154811363), ('sekolah', 0.034388668476263144), ('anak', 0.025951553801539184), ('murid', 0.02465808151796071), ('apa', 0.0203706611558914

In [25]:
for period, result in results.items():
    fig = result['model'].visualize_hierarchy(top_n_topics=10).show()
    fig.update_layout(title_text=f"Topics Over Time for {period}")
    fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [23]:
import nbformat

print(nbformat.__version__)

5.10.4


In [ ]:
for period, result in results.items():
    fig = result['model'].visualize_topics_over_time(result['topics_over_time'], top_n_topics=10)
    fig.update_layout(title_text=f"Topics Over Time for {period}")
    fig.show()

In [ ]:
top_topics = []


for period, result in results.items():
    top_10_topics = result['topic_info'][0:11]
    # top_10_topics['Name'] = top_10_topics['Name'].str.replace(r'^\d+_', '', regex=True)
    topic_descriptions = top_10_topics['Name'].tolist()
    top_topics.append({'period': period, 'topics': topic_descriptions})


top_topics_df = pd.DataFrame(top_topics)
display(top_topics_df)

,period,topics
0,2019Q4,"[-1_menteri_nadiem_ajar_merdeka, 0_empat_pokok..."
1,2020Q1,"[0_islam_minta_cegah_pemda, 1_merdeka_menteri_..."
2,2020Q2,"[-1_papua_ajar_perintah_covid, 0_mahasiswa_sek..."
3,2020Q3,"[0_kuat_kartu_prakerja_efektivitas, 1_papua_ot..."
4,2020Q4,"[0_islam_gtgt_cukup_seluruh, 1_menteri_ajar_pe..."
5,2021Q1,"[-1_akmrtv_sarah_radio_tv, 0_musik_tradisional..."
6,2021Q2,"[-1_hak_awal_anak_milik, 0_papua_otsus_infrast..."
7,2021Q3,"[-1_masyarakat_sehat_sanggau_papua, 0_pandemi_..."
8,2021Q4,"[0_tara_disabilitas_dorong_risma, 1_kota_narko..."
9,2022Q1,"[-1_ajar_bawaslu_milu_ptm, 0_tingkat_sehat_per..."


In [ ]:
for quarter in top_topics:
    print(f"{quarter['period']}: {quarter['topics']}")

2019Q4: ['-1_menteri_nadiem_ajar_merdeka', '0_empat_pokok_tetap_merdeka', '1_indonesia_nadiem_makarim_presiden', '2_kembang_mendikbud_nadiem_jangan', '3_tri_baru_tingkat_kritik', '4_analisis_negara_suatu_rakyat', '5_guru_anak_sekolah_jadi', '6_budaya_menteri_program_makarim', '7_menteri_un_enggak_pak']
2020Q1: ['0_islam_minta_cegah_pemda', '1_merdeka_menteri_ajar_nadiem', '2_disease_coronavirus_darurat_masa', '3_virus_corona_sebar_covid']
2020Q2: ['-1_papua_ajar_perintah_covid', '0_mahasiswa_sekolah_jadi_buat', '1_menteri_indonesia_ajar_budaya', '2_pandemi_masa_covid_indonesia', '3_darurat_sebar_timur_jawa', '4_new_normal_hadap_jabar', '5_dpr_apresiasi_kali_komisi', '6_islam_normal_pesantren_agama', '7_saran_hendak_bila_implementasi', '8_dokter_siap_normal_era']
2020Q3: ['0_kuat_kartu_prakerja_efektivitas', '1_papua_otonomi_khusus_barat', '2_ajar_pjj_nilai_jarak', '3_ajar_menteri_pandemi_covid']
2020Q4: ['0_islam_gtgt_cukup_seluruh', '1_menteri_ajar_perintah_indonesia']
2021Q1: ['-1_ak

# Evaluation

In [ ]:
coherence_obj = {}

for period, result in results.items():
    documents = pd.DataFrame({"Document": result['text'],
                              "ID": range(len(result['text'])),
                              "Topic": result['topics'],
                              })

    documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
    cleaned_docs = result['model']._preprocess_text(documents_per_topic.Document.values)

    vectorizer = result['model'].vectorizer_model
    analyzer = vectorizer.build_analyzer()

    words = vectorizer.get_feature_names_out()
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topic_words = [[words for words, _ in result['model'].get_topic(topic)] for topic in range(len(set(result['topics']))-1)]

    coherence_model_c_v = CoherenceModel(topics=topic_words,
                                    texts=tokens,
                                    corpus=corpus,
                                    dictionary=dictionary,
                                    coherence='c_v')
    coherence_model_u_mass = CoherenceModel(topics=topic_words,
                                    texts=tokens,
                                    corpus=corpus,
                                    dictionary=dictionary,
                                    coherence='u_mass')
    coherence_c_v = coherence_model_c_v.get_coherence()
    coherence_u_mass = coherence_model_u_mass.get_coherence()
    coherence_obj[period] = {
        'coherence_c_v': coherence_c_v,
        'coherence_u_mass': coherence_u_mass,
    }

In [ ]:
coherence_obj

{Period('2019Q4', 'Q-DEC'): {'coherence_c_v': 0.5586950531695594,
  'coherence_u_mass': -0.5607027727226086},
 Period('2020Q1', 'Q-DEC'): {'coherence_c_v': 0.7348581618075546,
  'coherence_u_mass': -0.16566044331697286},
 Period('2020Q2', 'Q-DEC'): {'coherence_c_v': 0.6084942290321547,
  'coherence_u_mass': -0.40894767149817063},
 Period('2020Q3', 'Q-DEC'): {'coherence_c_v': 0.6559972028358545,
  'coherence_u_mass': -0.07837524280155266},
 Period('2020Q4', 'Q-DEC'): {'coherence_c_v': 0.7809663259471452,
  'coherence_u_mass': -0.015403270677732079},
 Period('2021Q1', 'Q-DEC'): {'coherence_c_v': 0.7049338235698431,
  'coherence_u_mass': -0.42268877879561967},
 Period('2021Q2', 'Q-DEC'): {'coherence_c_v': 0.3430110023832821,
  'coherence_u_mass': -0.09053185699700497},
 Period('2021Q3', 'Q-DEC'): {'coherence_c_v': 0.7318491948466705,
  'coherence_u_mass': -0.5081136275309767},
 Period('2021Q4', 'Q-DEC'): {'coherence_c_v': 0.7626448785739149,
  'coherence_u_mass': -0.2946078328114052},
 Pe